# Fishoal project - ENSEEIHT

Shoal of fish modelisation with different behaviors


In [1]:
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np

from fishes import Fish
from aquarium import Aquarium2D, AquariumKDTree2D, Aquarium3D, AquariumKDTree2D, AquariumKDTree3D
from behaviors import StraightToroidalBehavior, StraightReboundBehavior, StraightVariableReboundBehavior, RandomBehavior, TrafalgarBehavior, AokiBehavior

%matplotlib widget

In [2]:
## SIMULATION PARAMETERS
frame_delay = 10                            # delay between frames in ms
dt_simu = 2                                 # time step of simulation in ms
width = 1000                                # width of aquarium
height = 1000                               # height of aquarium 
depth = 1000                                # depth of aquarium (only for 3D simulation)
nb_fish = 100                                # number of fishes in simulation
fish_color = "orange"                       # color of fishes  

steve_orange_img = plt.imread("./../ressources/steve_orange.png")
steve_red_img = plt.imread("./../ressources/steve_red.png")
steve_green_img = plt.imread("./../ressources/steve_green.png")

# PART I : Random behavior

## Version 1 : Random initial position and velocity with rebound

Fishes are added with random position and random velocity vector (normalized). They move straight at constant speed and rebound on aquarium edges.

### 2D

In [3]:
# Create the fishes in a 2D aquarium 
aquarium = Aquarium2D(width, height, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(2) * [width, height]
    velocity = (np.random.rand(2) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    behavior = StraightReboundBehavior(aquarium)
    fish = Fish(position, velocity, fish_color, behavior)
    aquarium.put_fish(i, fish)

### 3D

In [6]:
# Create the fishes in a 3D aquarium 
aquarium = Aquarium3D(width, height, depth, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(3) * [width, height, depth]
    velocity = (np.random.rand(3) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    behavior = StraightVariableReboundBehavior(aquarium, np.pi/2, 50)
    fish = Fish(position, velocity, fish_color, behavior)
    aquarium.put_fish(i, fish)

## Version 2 : Random initial position and velocity with toroidal behavior

Fishes are added with random position and random velocity vector (normalized). They move straight at constant speed and have toroidal behavior on aquarium edges.

### 2D

In [3]:
# Create the fishes in a 2D aquarium 
aquarium = Aquarium2D(width, height, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(2) * [width, height]
    velocity = (np.random.rand(2) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    behavior = StraightToroidalBehavior(aquarium)
    fish = Fish(position, velocity, fish_color, behavior)
    aquarium.put_fish(i, fish)

### 3D

In [9]:
# Create the fishes in a 3D aquarium 
aquarium = Aquarium3D(width, height, depth, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(3) * [width, height, depth]
    velocity = (np.random.rand(3) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    behavior = StraightToroidalBehavior(aquarium)
    fish = Fish(position, velocity, fish_color, behavior)
    aquarium.put_fish(i, fish)

## Version 3 : Random changes of velocity and direction

Fishes are added with random position and random velocity vector (normalized). They move straight at constant speed but can change direction with a certain probability. They rebound on aquarium edges.

### 2D

In [5]:
direction_change_prob = 0.1

# Create the fishes in a 2D aquarium
aquarium = Aquarium2D(width, height, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(2) * [width, height]
    velocity = (np.random.rand(2) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    behavior = RandomBehavior(aquarium, direction_change_prob)
    fish = Fish(position, velocity, fish_color, behavior)
    aquarium.put_fish(i, fish)

### 3D

In [3]:
direction_change_prob = 0.1

# Create the fishes in a 3D aquarium
aquarium = Aquarium3D(width, height, depth, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(3) * [width, height, depth]
    velocity = (np.random.rand(3) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    behavior = RandomBehavior(aquarium, direction_change_prob)
    fish = Fish(position, velocity, fish_color, behavior)
    aquarium.put_fish(i, fish)

# PART II : Behavior propagation : Trafalgar effect

Fishes are added with random position and random velocity vector (normalized). They move globally straight at constant speed but have slight variations from their base velocity to make them more real. The fishes rebound on aquarium edges.

A leader is chosen and has it's own behavior which is an amelioration of the **StraightReboundBehavior** called **StraightVariableReboundBehavior** for making it more real.

The other fishes can become contaminated when they are too close to the leader or another contaminated fish. When it happens, their base velocity becomes the same as the base velocity of the leader.

### 2D

In [12]:
max_angle_rand_variation = np.pi/12
delay_rand_variation = 50
contamination_dist = 10
delay_contamination = 5
contaminated_color = "green"

# Create the fishes in a 2D aquarium
aquarium = Aquarium2D(width, height, nb_fish, dt_simu)

# Create the leader fish
leader_behavior = StraightVariableReboundBehavior(
    aquarium,
    max_angle_rand_variation,
    delay_rand_variation)
velocity = (np.random.rand(2) * 2 - 1)
velocity /= np.linalg.norm(velocity)
position = np.random.rand(2) * [width, height]
fish_leader = Fish(position, velocity, "red", leader_behavior, steve_red_img)
aquarium.put_fish(0, fish_leader)

# Create the follower fishes
for i in range(1, nb_fish):
    position = np.random.rand(2) * [width, height]
    velocity = (np.random.rand(2) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    trafalgar_behavior = TrafalgarBehavior(
        aquarium,
        fish_leader,
        contamination_dist,
        max_angle_rand_variation,
        delay_rand_variation,
        delay_contamination,
        contaminated_color,
        contaminated_img=steve_green_img)
    fish = Fish(position, velocity, fish_color, trafalgar_behavior, steve_orange_img)
    aquarium.put_fish(i, fish)

### 3D

In [14]:
max_angle_rand_variation = np.pi/12
delay_rand_variation = 50
contamination_dist = 80
delay_contamination = 5
contaminated_color = "green"

# Create the fishes in a 2D aquarium
aquarium = Aquarium3D(width, height, depth, nb_fish, dt_simu)

# Create the leader fish
leader_behavior = StraightVariableReboundBehavior(
    aquarium,
    max_angle_rand_variation,
    delay_rand_variation)
velocity = (np.random.rand(3) * 2 - 1)
velocity /= np.linalg.norm(velocity)
position = np.random.rand(3) * [width, height, depth]
fish_leader = Fish(position, velocity, "red", leader_behavior, steve_red_img)
aquarium.put_fish(0, fish_leader)

# Create the follower fishes
for i in range(1, nb_fish):
    position = np.random.rand(3) * [width, height, depth]
    velocity = (np.random.rand(3) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    trafalgar_behavior = TrafalgarBehavior(
        aquarium,
        fish_leader,
        contamination_dist,
        max_angle_rand_variation,
        delay_rand_variation,
        delay_contamination,
        contaminated_color,
        contaminated_img=steve_green_img)
    fish = Fish(position, velocity, fish_color, trafalgar_behavior, steve_orange_img)
    aquarium.put_fish(i, fish)

# PART III : Aoki's behavior rules



### 2D

In [3]:
r_repulsion = 25
r_alignement = 30
r_attraction = 40
k_repulsion = 1
k_attraction = 0.005

# Create the fishes in a 2D aquarium
aquarium = AquariumKDTree2D(width, height, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(2) * [width, height]
    velocity = (np.random.rand(2) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    aoki_behavior = AokiBehavior(aquarium, r_repulsion, r_alignement, r_attraction, k_repulsion, k_attraction)
    fish = Fish(position, velocity, fish_color, aoki_behavior)
    aquarium.put_fish(i, fish)

### 3D

In [7]:
r_repulsion = 25
r_alignement = 30
r_attraction = 40
k_repulsion = 1
k_attraction = 1

# Create the fishes in a 2D aquarium
aquarium = AquariumKDTree3D(width, height, depth, nb_fish, dt_simu)
for i in range(0, nb_fish):
    position = np.random.rand(3) * [width, height, depth]
    velocity = (np.random.rand(3) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    aoki_behavior = AokiBehavior(aquarium, r_repulsion, r_alignement, r_attraction, k_repulsion, k_attraction)
    fish = Fish(position, velocity, fish_color, aoki_behavior)
    aquarium.put_fish(i, fish)

# Vizualization of the simulation in 2D or 3D

## 2D

In [ ]:
# Launch the simulation in 2D
fig, ax = plt.subplots()
xs = [fish.position[0] for fish in aquarium.fishes]
ys = [fish.position[1] for fish in aquarium.fishes]
points = ax.scatter(xs, ys, marker='o', c=fish_color)
ax.set_xlim(0, width)
ax.set_ylim(0, height)

def update(data):
    ys, xs, fish_colors = data
    points.set_offsets(np.c_[xs, ys])
    points.set_color(fish_colors)
    return points

def generate_points():
    while True:
        aquarium.update_all()
        xs = [fish.position[0] for fish in aquarium.fishes]
        ys = [fish.position[1] for fish in aquarium.fishes]
        fish_colors = [fish.color for fish in aquarium.fishes]
        yield (ys, xs, fish_colors)

anim = animation.FuncAnimation(fig, update, generate_points, interval=frame_delay, cache_frame_data=False)
plt.show()

## 3D

In [ ]:
# Launch the simulation in 3D
xs = [fish.position[0] for fish in aquarium.fishes]
ys = [fish.position[1] for fish in aquarium.fishes]
zs = [fish.position[2] for fish in aquarium.fishes]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(0, width)
ax.set_ylim(0, height)
ax.set_zlim(0, depth)

points = ax.scatter(xs, ys, zs, marker='o', c=fish_color, s=50)

def update(data):
    ys, xs, zs, fish_colors = data
    points._offsets3d = (xs, ys, zs)
    points.set_color(fish_colors)
    return points

def generate_points():
    while True:
        aquarium.update_all()
        xs = [fish.position[0] for fish in aquarium.fishes]
        ys = [fish.position[1] for fish in aquarium.fishes]
        zs = [fish.position[2] for fish in aquarium.fishes]
        fish_colors = [fish.color for fish in aquarium.fishes]
        yield (ys, xs, zs, fish_colors)

anim = animation.FuncAnimation(fig, update, generate_points, interval=frame_delay, cache_frame_data=False)
plt.show()

# FUNNY STEVE BONUS

"Steve, OuuOuOouuOOu, le poisson steve, il est oraaannnnngeee"

Here is a funny simulation with Trafalgar behavior but with images instead of points.
It is super slow so take your time.

In [ ]:
# Parameters
save_path = "./../results/simulation_steve.mp4"
save_result = False
max_angle_rand_variation = np.pi/12
delay_rand_variation = 50
contamination_dist = 30
delay_contamination = 5
contaminated_color = "green"

# Create the fishes in a 2D aquarium
aquarium = Aquarium2D(width, height, nb_fish, dt_simu)

# Create the leader fish with random initial position and velocity
leader_behavior = StraightVariableReboundBehavior(
    aquarium,
    max_angle_rand_variation,
    delay_rand_variation)
velocity = (np.random.rand(2) * 2 - 1)
velocity /= np.linalg.norm(velocity)
position = np.random.rand(2) * [width, height]
fish_leader = Fish(position, velocity, "red", leader_behavior, steve_red_img)
aquarium.put_fish(0, fish_leader)

# Create the follower fishes with random initial position and velocity
for i in range(1, nb_fish):
    position = np.random.rand(2) * [width, height]
    velocity = (np.random.rand(2) * 2 - 1)
    velocity /= np.linalg.norm(velocity)
    trafalgar_behavior = TrafalgarBehavior(
        aquarium,
        fish_leader,
        contamination_dist,
        max_angle_rand_variation,
        delay_rand_variation,
        delay_contamination,
        contaminated_color,
        contaminated_img=steve_green_img)
    fish = Fish(position, velocity, fish_color, trafalgar_behavior, steve_orange_img)
    aquarium.put_fish(i, fish)

# Launch the simulation in 2D
fig, ax = plt.subplots()
ax.set_xlim(0, width)
ax.set_ylim(0, height)

fish_boxes = []
for fish in aquarium.fishes:
    image_box = OffsetImage(fish.image, zoom=0.05)
    ab = AnnotationBbox(image_box, (fish.position[0], fish.position[1]), frameon=False)
    ax.add_artist(ab)
    fish_boxes.append(ab)

def update(data):
    xs, ys, fish_images = data
    for i, ab in enumerate(fish_boxes):
        ab.offsetbox = OffsetImage(fish_images[i], zoom=0.05)
        ab.xy = (xs[i], ys[i])
        ab.xybox = (xs[i], ys[i])
    return fish_boxes

def generate_points():
    while True:
        aquarium.update_all()
        xs = [fish.position[0] for fish in aquarium.fishes]
        ys = [fish.position[1] for fish in aquarium.fishes]
        fish_images = [fish.image for fish in aquarium.fishes]
        yield (xs, ys, fish_images)

ani = animation.FuncAnimation(fig, update, generate_points, interval=frame_delay, cache_frame_data=False, blit=False)
plt.show()

if save_result:
    ani.save("./../results/simulation_steve.mp4", writer="ffmpeg", fps=30)